<a href="https://colab.research.google.com/github/JooEHyeon/SaltLux_Chatbot/blob/master/apparel_advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

my_path = '/content/notebooks'
# Colab Notebooks 안에 my_env 폴더에 패키지 저장
os.symlink('/content/drive/My Drive/Colab Notebooks/my_env', my_path)
sys.path.insert(0, my_path)


Mounted at /content/drive


In [ ]:
from keras import models
from keras import layers
from keras import optimizers, losses, metrics
from keras import preprocessing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

from konlpy.tag import Okt

<br>
<br>
# 데이터 로드

Seq2Seq에서의 임베딩이 이전 예제와 다른 점은 아래와 같이 태그를 사용한다는 것입니다. 임베딩의 0~3번째에 각각 PADDING, START, END, OOV 태그를 넣습니다. 사실 그냥 똑같은 단어라고 보시면 됩니다. 다만 이 단어들이 Seq2Seq의 동작을 제어합니다. 

예를 들어, 디코더 입력에 START가 들어가면 디코딩의 시작을 의미합니다. 반대로 디코더 출력에 END가 나오면 디코딩을 종료합니다.
<br>
<br>
<br>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
# 태그 단어
PAD = "<PADDING>"   # 패딩
STA = "<START>"     # 시작
END = "<END>"       # 끝
OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)

# 태그 인덱스
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

# 데이터 타입
ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수
max_sequences = 30

# 임베딩 벡터 차원
embedding_dim = 100

# LSTM 히든레이어 차원
lstm_hidden_dim = 128

# 정규 표현식 필터
RE_FILTER = re.compile("[.,!?\"':;~()]")

# # 챗봇 데이터 로드
# chatbot_data = pd.read_csv('./dataset/chatbot/ChatbotData.csv', encoding='utf-8')
# question, answer = list(chatbot_data['Q']), list(chatbot_data['A'])

In [ ]:
wear_data = pd.read_csv("/content/drive/My Drive/자연어처리,추천/NLPer/chatbot_pra/wear.csv")
print(wear_data.shape)
customer = wear_data[wear_data.SPEAKER == "고객"].SENTENCE
store = wear_data[wear_data.SPEAKER == "점원"].SENTENCE
print(customer.shape, store.shape) # 질문의 개수와 답의 개수가 일치하지 않는다.
store.head()
print(type(wear_data))

(15826, 20)
(8381,) (7445,)
<class 'pandas.core.frame.DataFrame'>


In [ ]:
prev = "고객"
store_arr = []
customer_arr = []
store_stc = ""
customer_stc = ""

for i in range(wear_data.shape[0]):
    if (prev == wear_data.iloc[i].SPEAKER):
        if prev == "점원":
             store_stc += (" "+wear_data.iloc[i].SENTENCE)
        else : 
            customer_stc += (" "+wear_data.iloc[i].SENTENCE)
            
    elif prev == "점원":
        store_arr.append(store_stc)
        customer_stc = wear_data.iloc[i].SENTENCE
        prev = "고객"
    else :
        customer_arr.append(customer_stc)
        store_stc = wear_data.iloc[i].SENTENCE
        prev = "점원"

print(len(store_arr))
print(len(customer_arr))
print(store_arr[-1])
print(customer_arr[-1]) # 자료 상에서 이후에는 계속 고객의 물음만 계속된다. 코드 레벨에서 이 부분은 빼게 구현했다. (stc는 만들어지지만 arr에 append 안하게 된다.)

7301
7301
요즘 파스텔 톤이 유행이에요
요즘 유행하는 색깔이 뭐예요?


In [ ]:
question = []
answer = []

for Q in customer_arr:
    question.append(Q.replace("[^\w]", " "))

for A in store_arr:
    answer.append(A.replace("[^\w]", " "))


챗봇의 훈련을 위해서 송영숙님이 공개한 한글 데이터셋을 로드합니다. 질문과 대답, 감정 등 총 3개의 항목으로 구성되어 있습니다. 감정 분류는 Seq2Seq에 필요가 없기 때문에 사용하지 않습니다.

https://github.com/songys/Chatbot_data
<br>
<br>
<br>

In [ ]:
# 데이터 개수
len(question), len(answer)


(7301, 7301)

In [ ]:
# 챗봇 데이터 출력
for i in range(10):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()


Q :  신발은 여기 있는 게 다예요?
A : 네 성인이나 아동 다 있어요 발 사이즈 몇 신으세요?

Q : 230이요
A : 편하게 신을 수 있는 거 찾으세요?

Q : 네 봄이니까 편하게 신을 수 있는 거
A : 이런 건 어떠세요? 이런 거도 신발 무척 편하거든요

Q : 굽 좀 높은 거 없나요?
A : 봄 상품은 아직 어른 제품이 많이 안나왔습니다

Q : 언제 들어와요?
A : 이번주 지나면 들어올 거예요

Q : 이거는 가죽이에요?
A : 가죽 아니고 쎄무예요

Q : 가죽은 얼마예요?
A : 2만 9천 원입니다

Q : 털 달린 거 저거는 사이즈 있어요?
A : 230이 없어요  이거 한 번 신어보세요

Q : 좀 크네 또 안 들어와요?
A : 네 이건 다 끝났어요

Q : 가방 매는 거 보고 있어요
A : 여기 있어요



<br>
<br>
# 단어 사전 생성

In [ ]:
# 형태소분석 함수
def pos_tag(sentences):
    
    # KoNLPy 형태소분석기 설정
    tagger = Okt()
    
    # 문장 품사 변수 초기화
    sentences_pos = []
    
    # 모든 문장 반복
    for sentence in sentences:
        # 특수기호 제거
        sentence = re.sub(RE_FILTER, "", sentence)
        
        # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
        sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)
        
    return sentences_pos

문장을 먼저 최소 단위인 토큰으로 나누어야 합니다. 한글 문장이라면 보통 KoNLPy(코엔엘파이)를 통해 형태소분석을 적용합니다. Hannanum, Kkma, Komoran, Mecab, Okt(Twitter) 등 여러가지 라이브러리를 동일한 인터페이스로 실행이 가능합니다.

KoNLPy는 자바로 구현이 되어 있습니다. 맥OS와 달리 윈도우 아나콘다는 아래와 같은 작업을 거쳐야 합니다.

< 1. 자바 설치 ><br>
커맨드창에서 'java -version' 명령어를 실행하여 버전이 1.7 이상이어야 합니다.

< 2. JPype1 설치 ><br>
파이썬에서 자바를 호출할 수 있는 라이브러리입니다.

< 3. KoNLPy 설치 ><br>
위의 두 가지가 미리 완료되어 있어야만 가능합니다.

아래 링크에서 좀 더 자세한 방법을 확인하시기 바랍니다.<br>
-> https://ericnjennifer.github.io/python_visualization/2018/01/21/PythonVisualization_Chapt1.html
<br>
<br>
<br>

In [ ]:
# 형태소분석 수행
question = pos_tag(question)
answer = pos_tag(answer)

# 형태소분석으로 변환된 챗봇 데이터 출력
for i in range(10):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()


Q : 신발 은 여기 있는 게 다예 요
A : 네 성인 이나 아동 다 있어요 발 사이즈 몇 신으세요

Q : 230 이요
A : 편하게 신 을 수 있는 거 찾으세요

Q : 네 봄 이니까 편하게 신 을 수 있는 거
A : 이런 건 어떠세요 이런 거도 신발 무척 편하거든요

Q : 굽 좀 높은 거 없나요
A : 봄 상품 은 아직 어른 제품 이 많이 안 나왔습니다

Q : 언제 들어와요
A : 이번 주 지나면 들어올 거 예요

Q : 이 거 는 가죽 이에요
A : 가죽 아니고 쎄 무예 요

Q : 가죽 은 얼마 예요
A : 2만 9천 원 입니다

Q : 털 달린 거 저 거 는 사이즈 있어요
A : 230 이 없어요 이 거 한 번 신어 보세요

Q : 좀 크네 또 안 들어와요
A : 네 이건 다 끝났어요

Q : 가방 매는 거 보고 있어요
A : 여기 있어요



In [ ]:
import pickle

# save
with open('question.pickle', 'wb') as f:
    pickle.dump(question, f, pickle.HIGHEST_PROTOCOL)

with open('answer.pickle', 'wb') as f:
    pickle.dump(answer, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
from keras import models
from keras import layers
from keras import optimizers, losses, metrics
from keras import preprocessing


with open('/content/drive/My Drive/자연어처리,추천/NLPer/chatbot_pra/pickle/answer_j.pickle', 'rb') as f:
    answer = pickle.load(f)

with open('/content/drive/My Drive/자연어처리,추천/NLPer/chatbot_pra/pickle/question_j.pickle', 'rb') as f:
    question = pickle.load(f)

In [ ]:
tokenized_Q = question
tokenized_A = answer
print(len(tokenized_Q))

7301


In [ ]:
from gensim.models import Word2Vec
model_Q = Word2Vec(sentences = tokenized_Q, size = 100, window = 4, min_count = 3, workers = 4, sg = 0)
model_Q.wv.vectors.shape

(635, 100)

In [ ]:
from gensim.models import Word2Vec
model_A = Word2Vec(sentences = tokenized_A, size = 100, window = 4, min_count = 3, workers = 4, sg = 0)
model_A.wv.vectors.shape

(637, 100)

In [ ]:
model_Q.save('w2v_Q.pickle')
model_A.save('w2v_A.pickle')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
# 질문과 대답 문장들을 하나로 합침
sentences = []
sentences.extend(question)
sentences.extend(answer)

words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

# 길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

# 중복된 단어 삭제
words = list(set(words))

# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, STA, END, OOV]

질문과 대답 문장들을 합쳐서 전체 단어 사전을 만듭니다. 자연어처리에서는 항상 이렇게 단어를 인덱스에 따라 정리를 해야 합니다. 그래야지 문장을 인덱스 배열로 바꿔서 임베딩 레이어에 넣을 수 있습니다. 또한 모델의 출력에서 나온 인덱스를 다시 단어로 변환하는데도 필요합니다.
<br>
<br>
<br>

In [ ]:
# 단어 개수
len(words)


6403

In [ ]:
# 단어 출력
words[:20]


['<PADDING>',
 '<START>',
 '<END>',
 '<OOV>',
 '간단한',
 '하는고',
 '든다',
 '거시',
 '별도',
 '해드릴께요',
 '가타',
 '할수는',
 '좋아하시겠네요',
 '총',
 '그렇구나',
 '9시',
 '수박',
 '어린',
 '나오네요',
 '시내']

In [ ]:
# 단어와 인덱스의 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

In [ ]:
# 단어 -> 인덱스
# 문장을 인덱스로 변환하여 모델 입력으로 사용
dict(list(word_to_index.items())[:20])


{'9시': 15,
 '<END>': 2,
 '<OOV>': 3,
 '<PADDING>': 0,
 '<START>': 1,
 '가타': 10,
 '간단한': 4,
 '거시': 7,
 '그렇구나': 14,
 '나오네요': 18,
 '든다': 6,
 '별도': 8,
 '수박': 16,
 '시내': 19,
 '어린': 17,
 '좋아하시겠네요': 12,
 '총': 13,
 '하는고': 5,
 '할수는': 11,
 '해드릴께요': 9}

In [ ]:
# 인덱스 -> 단어
# 모델의 예측 결과인 인덱스를 문장으로 변환시 사용
dict(list(index_to_word.items())[:20])


{0: '<PADDING>',
 1: '<START>',
 2: '<END>',
 3: '<OOV>',
 4: '간단한',
 5: '하는고',
 6: '든다',
 7: '거시',
 8: '별도',
 9: '해드릴께요',
 10: '가타',
 11: '할수는',
 12: '좋아하시겠네요',
 13: '총',
 14: '그렇구나',
 15: '9시',
 16: '수박',
 17: '어린',
 18: '나오네요',
 19: '시내'}

<br>
<br>
# 전처리

In [ ]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type): 
    
    sentences_index = []
    
    # 모든 문장에 대해서 반복
    for sentence in sentences:
        sentence_index = []
        
        # 디코더 입력일 경우 맨 앞에 START 태그 추가
        if type == DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])
        
        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                # 사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                # 사전에 없는 단어면 OOV 인덱스를 추가
                sentence_index.extend([vocabulary[OOV]])

        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 END 태그 추가
            if len(sentence_index) >= max_sequences:
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
            else:
                sentence_index += [vocabulary[END]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]
            
        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]
        
        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)

    return np.asarray(sentences_index)

Seq2Seq에서는 학습시 다음과 같이 총 3개의 데이터가 필요합니다.

인코더 입력 : 12시 땡<br>
디코더 입력 : START 하루 가 또 가네요<br>
디코더 출력 : 하루 가 또 가네요 END

원래 Seq2Seq는 디코더의 현재 출력이 디코더의 다음 입력으로 들어갑니다. 다만 학습에서는 굳이 이렇게 하지 않고 디코더 입력과 디코더 출력의 데이터를 각각 만듭니다. 

그러나 예측시에는 이런 방식이 불가능합니다. 출력값을 미리 알지 못하기 때문에, 디코더 입력을 사전에 생성할 수가 없습니다. 이런 문제를 해결하기 위해 훈련 모델과 예측 모델을 따로 구성해야 합니다. 모델 생성 부분에서 다시 자세히 설명을 드리겠습니다.
<br>
<br>
<br>

In [ ]:
# 인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)

# 첫 번째 인코더 입력 출력 (12시 땡)
x_encoder[0]


array([2433, 5267,   36, 4280, 3254,  106,  660,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [ ]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)

# 첫 번째 디코더 입력 출력 (START 하루 가 또 가네요)
x_decoder[0]


array([   1, 5816, 2312, 1635, 1460, 1506, 5292, 4857, 3251, 6184, 5761,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [ ]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(answer, word_to_index, DECODER_TARGET)

# 첫 번째 디코더 목표 출력 (하루 가 또 가네요 END)
y_decoder[0]


array([5816, 2312, 1635, 1460, 1506, 5292, 4857, 3251, 6184, 5761,    2,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [ ]:
# 원핫인코딩 초기화
one_hot_data = np.zeros((len(y_decoder), max_sequences, len(words)))

# 디코더 목표를 원핫인코딩으로 변환
# 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
for i, sequence in enumerate(y_decoder):
    for j, index in enumerate(sequence):
        one_hot_data[i, j, index] = 1

# 디코더 목표 설정
y_decoder = one_hot_data

# 첫 번째 디코더 목표 출력
y_decoder[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

인코더 입력과 디코더 입력은 임베딩 레이어에 들어가는 인덱스 배열입니다. 반면에 디코더 출력은 원핫인코딩 형식이 되어야 합니다. 디코더의 마지막 Dense 레이어에서 softmax로 나오기 때문입니다.

<br>
<br>
# 모델 생성

In [ ]:
#--------------------------------------------
# 훈련 모델 인코더 정의
#--------------------------------------------

# 입력 문장의 인덱스 시퀀스를 입력으로 받음
encoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
encoder_outputs = layers.Embedding(len(words), embedding_dim)(encoder_inputs)

# return_state가 True면 상태값 리턴
# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
encoder_outputs, state_h, state_c = layers.LSTM(lstm_hidden_dim,
                                                dropout=0.1,
                                                recurrent_dropout=0.5,
                                                return_state=True)(encoder_outputs)

# 히든 상태와 셀 상태를 하나로 묶음
encoder_states = [state_h, state_c]



#--------------------------------------------
# 훈련 모델 디코더 정의
#--------------------------------------------

# 목표 문장의 인덱스 시퀀스를 입력으로 받음
decoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
decoder_embedding = layers.Embedding(len(words), embedding_dim)
decoder_outputs = decoder_embedding(decoder_inputs)

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임 스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리하기 위함
decoder_lstm = layers.LSTM(lstm_hidden_dim,
                           dropout=0.1,
                           recurrent_dropout=0.5,
                           return_state=True,
                           return_sequences=True)

# initial_state를 인코더의 상태로 초기화
decoder_outputs, _, _ = decoder_lstm(decoder_outputs,
                                     initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = layers.Dense(len(words), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)



#--------------------------------------------
# 훈련 모델 정의
#--------------------------------------------

# 입력과 출력으로 함수형 API 모델 생성
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])    

지금까지의 예제는 Sequential 방식의 모델이었습니다. 하지만 이번에는 함수형 API 모델을 사용했습니다. 인코더와 디코더가 따로 분리되어야 하는데, 단순히 레이어를 추가하여 붙이는 순차형으로는 구현이 불가능하기 때문입니다. 

Model() 함수로 입력과 출력을 따로 설정하여 모델을 만듭니다. 그다음 compile과 fit은 이전과 동일하게 적용하시면 됩니다.
<br>
<br>
<br>

In [ ]:
#--------------------------------------------
#  예측 모델 인코더 정의
#--------------------------------------------

# 훈련 모델의 인코더 상태를 사용하여 예측 모델 인코더 설정
encoder_model = models.Model(encoder_inputs, encoder_states)



#--------------------------------------------
# 예측 모델 디코더 정의
#--------------------------------------------

# 예측시에는 훈련시와 달리 타임 스텝을 한 단계씩 수행
# 매번 이전 디코더 상태를 입력으로 받아서 새로 설정
decoder_state_input_h = layers.Input(shape=(lstm_hidden_dim,))
decoder_state_input_c = layers.Input(shape=(lstm_hidden_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]    

# 임베딩 레이어
decoder_outputs = decoder_embedding(decoder_inputs)

# LSTM 레이어
decoder_outputs, state_h, state_c = decoder_lstm(decoder_outputs,
                                                 initial_state=decoder_states_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
decoder_states = [state_h, state_c]

# Dense 레이어를 통해 원핫 형식으로 각 단어 인덱스를 출력
decoder_outputs = decoder_dense(decoder_outputs)

# 예측 모델 디코더 설정
decoder_model = models.Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

예측 모델은 이미 학습된 훈련 모델의 레이어들을 그대로 재사용합니다. 예측 모델 인코더는 훈련 모델 인코더과 동일합니다. 그러나 예측 모델 디코더는 매번 LSTM 상태값을 입력으로 받습니다. 또한 디코더의 LSTM 상태를 출력값과 같이 내보내서, 다음 번 입력에 넣습니다. 

이렇게 하는 이유는 LSTM을 딱 한번의 타임 스텝만 실행하기 때문입니다. 그래서 매번 상태값을 새로 초기화 해야 합니다. 이와 반대로 훈련할때는 문장 전체를 계속 LSTM으로 돌리기 때문에 자동으로 상태값이 전달됩니다. 

<br>
<br>
# 훈련 및 테스트

In [ ]:
# 인덱스를 문장으로 변환
def convert_index_to_text(indexs, vocabulary): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break;
        if vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence.extend([vocabulary[OOV_INDEX]])
            
        # 빈칸 추가
        sentence += ' '

    return sentence

In [ ]:
# 에폭 반복
for epoch in range(20):
    print('Total Epoch :', epoch + 1)

   
    # 훈련 시작
    history = model.fit([x_encoder, x_decoder],
                        y_decoder,
                        epochs=10,
                        batch_size=64,
                        verbose=1)
    
    # 정확도와 손실 출력
    # print('accuracy :', history.history['acc'][-1])
    # print('loss :', history.history['loss'][-1])
    
    # 문장 예측 테스트
    # (3 박 4일 놀러 가고 싶다) -> (여행 은 언제나 좋죠)
    input_encoder = x_encoder[2].reshape(1, x_encoder[2].shape[0])
    input_decoder = x_decoder[2].reshape(1, x_decoder[2].shape[0])
    results = model.predict([input_encoder, input_decoder])
    
    # 결과의 원핫인코딩 형식을 인덱스로 변환
    # 1축을 기준으로 가장 높은 값의 위치를 구함
    indexs = np.argmax(results[0], 1) 
    
    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
    print(sentence)
    print()



Total Epoch : 1
Epoch 1/10
115/115 [==============================] - 24s 207ms/step - loss: 2.1391 - accuracy: 0.7737
Epoch 2/10
115/115 [==============================] - 24s 208ms/step - loss: 1.2797 - accuracy: 0.8146
Epoch 3/10
115/115 [==============================] - 24s 208ms/step - loss: 1.2154 - accuracy: 0.8189
Epoch 4/10
115/115 [==============================] - 24s 206ms/step - loss: 1.1803 - accuracy: 0.8216
Epoch 5/10
115/115 [==============================] - 24s 205ms/step - loss: 1.1485 - accuracy: 0.8251
Epoch 6/10
115/115 [==============================] - 24s 206ms/step - loss: 1.1180 - accuracy: 0.8290
Epoch 7/10
115/115 [==============================] - 24s 205ms/step - loss: 1.0890 - accuracy: 0.8331
Epoch 8/10
115/115 [==============================] - 24s 212ms/step - loss: 1.0610 - accuracy: 0.8369
Epoch 9/10
115/115 [==============================] - 24s 207ms/step - loss: 1.0351 - accuracy: 0.8398
Epoch 10/10
115/115 [==============================] - 24

In [ ]:
from keras.models import load_model
model.save('chatbot_model.h5')

학습이 진행될수록 예측 문장이 제대로 생성되는 것을 볼 수 있습니다. 다만 여기서의 예측은 단순히 테스트를 위한 것이라, 인코더 입력과 디코더 입력 데이터가 동시에 사용됩니다. 아래 문장 생성에서는 예측 모델을 적용하기 때문에, 오직 인코더 입력 데이터만 집어 넣습니다.

<br>
<br>
# 문장 생성

In [ ]:
from keras.models import load_model
model = load_model('/content/drive/My Drive/자연어처리,추천/NLPer/chatbot_pra/pickle/1024_save/chatbot_model.h5')

In [ ]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)
    
    return input_seq

In [ ]:
# 텍스트 생성
def generate_text(input_seq):
    
    # 입력을 인코더에 넣어 마지막 상태 구함
    states = encoder_model.predict(input_seq)

    # 목표 시퀀스 초기화
    target_seq = np.zeros((1, 1))
    
    # 목표 시퀀스의 첫 번째에 <START> 태그 추가
    target_seq[0, 0] = STA_INDEX
    
    # 인덱스 초기화
    indexs = []
    
    # 디코더 타임 스텝 반복
    while 1:
        # 디코더로 현재 타임 스텝 출력 구함
        # 처음에는 인코더 상태를, 다음부터 이전 디코더 상태로 초기화
        decoder_outputs, state_h, state_c = decoder_model.predict(
                                                [target_seq] + states)

        # 결과의 원핫인코딩 형식을 인덱스로 변환
        index = np.argmax(decoder_outputs[0, 0, :])
        indexs.append(index)
        
        # 종료 검사
        if index == END_INDEX or len(indexs) >= max_sequences:
            break

        # 목표 시퀀스를 바로 이전의 출력으로 설정
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = index
        
        # 디코더의 이전 상태를 다음 디코더 예측에 사용
        states = [state_h, state_c]

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
        
    return sentence

제일 첫 단어는 START로 시작합니다. 그리고 출력으로 나온 인덱스를 디코더 입력으로 넣고 다시 예측을 반복합니다. 상태값을 받아 다시 입력으로 같이 넣는 것에 주의하시기 바랍니다. END 태그가 나오면 문장 생성을 종료합니다.
<br>
<br>
<br>

In [ ]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('이 티셔츠 사이즈 있나요')
input_seq


array([[4866,  317, 3251, 4466,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]])

In [ ]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


'세척 교복 이었고 레드 어머니 버튼 딱딱해서 금도 뛰어난 온도 블랙 버스 팝니다 괜찮은가요 뛰어난 가까운 뛰어난 여가 였죠 세계지도 물 신으실라면 굉장히 굉장히 굉장히 굉장히 등하교 등하교 군데 군데 '

데이터셋에 있는 문장과 똑같은 입력을 넣으니, 역시 정확히 일치하는 답변이 출력되었습니다.
<br>
<br>
<br>

In [ ]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('이 티셔츠 사이즈 있나요')
input_seq


array([[ 76,   3, 380,   8,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [ ]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


'네 있어요 '

데이터셋 문장에서는 없던 '같이'라는 단어를 추가해 보았습니다. 그래도 비슷한 의미란 것을 파악하여 동일한 답변이 나왔습니다.
<br>
<br>
<br>

In [ ]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('교환 가능한가요?')
input_seq


array([[458, 263,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [ ]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


'노트북 넣어 다니시려면 이 정도 는 하셔야 해 요 '

In [ ]:
example = input()
input_seq = make_predict_input(example)
sentence = generate_text(input_seq)
sentence

이 가방 한정판으로 나온거에요


'재질 가방 은 따로 찾으시는 건 있으세요 '

하지만 데이터셋에 없던 '가려고'로 입력을 수정하니, 전혀 다른 문장이 출력되었습니다. 이는 우리가 데이터의 일부인 100개 문장만 학습했기 때문입니다. 데이터의 개수를 늘려서 훈련할수록 일반화 능력이 더욱 높아집니다.